In [6]:
import unittest
from unittest.mock import patch, MagicMock
import sys

sys.path.insert(1, "../utils/")
from uniprot_api import uniprot_request
from processDB import is_float

/home/michael/Desktop/ISDB/tests/../utils/processDB.py:256: SyntaxWarning: invalid escape sequence '\d'
  re.search("NCBI:\d+", x)[0].replace("NCBI:", "")
/home/michael/Desktop/ISDB/tests/../utils/processDB.py:257: SyntaxWarning: invalid escape sequence '\d'
  if (re.search("NCBI:\d+", x) is not None)
/home/michael/Desktop/ISDB/tests/../utils/processDB.py:263: SyntaxWarning: invalid escape sequence '\d'
  re.search("NCBI:\d+", x)[0].replace("NCBI:", "")
/home/michael/Desktop/ISDB/tests/../utils/processDB.py:264: SyntaxWarning: invalid escape sequence '\d'
  if (re.search("NCBI:\d+", x) is not None)
/home/michael/Desktop/ISDB/tests/../utils/processDB.py:737: SyntaxWarning: invalid escape sequence '\d'
  data["Parasite"] = data["Parasite"].apply(lambda x: re.sub(".\d", "", x))


AttributeError: 'NoneType' object has no attribute 'lower'

In [2]:
u = uniprot_request()

In [55]:
def clean_dataframe(dataFrame: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans dataframe: drops nan values in TaxIds and duplicate values
    """

    def join_unique(values: list) -> list:
        unique_values = np.unique([i for v in values for i in str(v).split("|")])
        return "|".join(unique_values)

    dataFrame = dataFrame.dropna(subset=["sourceTaxId", "targetTaxId"])
    # TODO include same - same mappings
    # dataFrame = dataFrame[dataFrame["sourceTaxId"] != dataFrame["targetTaxId"]]
    dataFrame = (
        dataFrame.groupby(["sourceTaxId", "targetTaxId", "sourceUid", "targetUid"])
        .agg(lambda x: join_unique(x))
        .reset_index()
    )
    # dataFrame = dataFrame.drop_duplicates(
    #     subset=["sourceTaxId", "targetTaxId", "sourceUid", "targetUid"]
    # )
    return dataFrame

In [98]:
import pandas as pd
import numpy as np
import requests
import xmltodict
t = pd.read_csv("../../data_subsample/gmpd_debug.tsv",sep="\t")

In [99]:
t

# u.tax_name_request("Wheat")

,parasiteName1,parasiteName2,hostName1,hostName2,interactionType,reference,hostName3,sourceTaxId,sourceName,targetTaxId,targetName,sourceUid,targetUid
0,2019-nCoV,Alphacoronavirus Alphacoronavirus 1,Homo Sapiens,Acinonyx jubatus,Virus,Kennedy et al. 2003,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
1,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Kennedy et al. 2003,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
2,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
3,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
4,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
5,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
6,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
7,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN
8,Feline coronavirus,Alphacoronavirus Alphacoronavirus 1,Acinonyx jubatus,Acinonyx jubatus,Virus,Munson et al. 2004,NaN,NaN,NaN,32536,Acinonyx jubatus (Cheetah),NaN,NaN


In [79]:
def send(name: str, fieldname: str = "SCIENTIFICNAME"):
        requestURL = "https://www.ebi.ac.uk/proteins/api/taxonomy/name/"
        requestURL += name.strip().replace(" ", "%20")
        requestURL += (
            "?pageNumber=1&pageSize=100&searchType=EQUALSTO&fieldName=" + fieldname
        )

        r = requests.get(requestURL, headers={"Accept": "application/xml"})

        if not r.ok:
            r.raise_for_status()
        return r.text

def _get_name(tax: dict) -> str:
    """Creates full taxonomical name using all annotated naming fields:
    Scientific name, Common name, Synonymes (see UniProt Taxonomy);

    Args:
        tax (dict): Dictionary holding all names

    Returns:
        str: Full taxonomical name
    """
    if "commonName" in tax.keys() and "synonym" in tax.keys():
        return f"{tax['scientificName']} ({tax['commonName']}) ({tax['synonym']})"
    elif "commonName" in tax.keys():
        return f"{tax['scientificName']} ({tax['commonName']})"
    elif "synonym" in tax.keys():
        return f"{tax['scientificName']} ({tax['synonym']})"
    else:
        return f"{tax['scientificName']}"

try:
    r = send()
except Exception as e:
    try:
        r = send(name, "COMMONNAME")
    except Exception as e:
        return None, None

tax = xmltodict.parse(r)["taxonomies"]["taxonomies"]["taxonomy"]
if type(tax) is list:
    # print(name)
    return None, None

name = self._get_name(tax)

id = tax["taxonomyId"]
return name, id

SyntaxError: 'return' outside function (3739597772.py, line 39)

In [90]:
t

,Host,Pathogen,Ref,targetTaxId,targetName,sourceTaxId,sourceName,sourceUid,targetUid
0,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
1,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
2,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
3,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
4,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
5,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
6,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
7,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN
8,Wheat,Fusarium graminearum,Astolfi et al. (2012),NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
dataFrame = dataFrame.astype(str)
dataFrame = (
    dataFrame.groupby(["sourceTaxId", "targetTaxId", "sourceUid", "targetUid"])
    .agg(lambda x: join_unique(x))
    .reset_index()
)
dataFrame

,sourceTaxId,targetTaxId,sourceUid,targetUid,level_0,index,sourceName,targetName,interactionType,ontology,reference,database
0,1000288,9615,nan,nan,3,3,Influenza A virus (A/canine/Zhejiang/1/2010(H3...,Canis lupus familiaris (Dog) (Canis familiaris),nan,nan,23107656|nan,BVBRC
1,1000289,9606,nan,nan,4,4,Influenza A virus (A/Korea/NAP-1/2009(H1N1)),Homo sapiens (Human),nan,nan,nan,BVBRC
2,100053,9365,nan,nan,0,0,Leptospira alexanderi,Erinaceus europaeus (Western European hedgehog),nan,nan,nan,BVBRC
3,100053,9606,nan,nan,1,1,Leptospira alexanderi,Homo sapiens (Human),nan,nan,Representative|nan,BVBRC
4,100053,9913,nan,nan,2,2,Leptospira alexanderi,Bos taurus (Bovine),nan,nan,nan,BVBRC
5,1000561,9606,nan,nan,5,5,Pseudomonas aeruginosa AES-1R,Homo sapiens (Human),nan,nan,nan,BVBRC
6,1005933,1975053,nan,nan,8,8,uncultured Methanoregula sp.,Urostomides striatus,nan,nan,nan,BVBRC
7,1005933,35101,nan,nan,7,7,uncultured Methanoregula sp.,Cyclidium porcatum,nan,nan,nan,BVBRC
8,1005933,5965,nan,nan,6,6,uncultured Methanoregula sp.,Heterometopus palaeformis,nan,nan,nan,BVBRC
